# AG2: Orchestrating agents (Swarm)

- Orchestrating agents<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/orchestrations
- Swarm<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/swarm
- Swarm Deep-dive **TODO:**<br>
  - https://docs.ag2.ai/docs/user-guide/advanced-concepts/swarm/deep-dive

## Orchestrating agents

- **Two-agent chat**: The simplest form of conversation pattern where two agents chat back-and-forth with each other. This has been demonstrated in the previous examples.
- **Sequential chat**: A sequence of chats, each between two agents, chained together by a carryover mechanism (which brings the summary of the previous chat to the context of the next chat). Useful for simple sequential workflows.
- **Group chat**: A chat with more than two agents with options on how agents are selected. See GroupChat Overview for further details.
- **Nested chat**: A mechanism to package a workflow into a single agent/chat for reuse in a workflow.
- **Swarm**: A pattern based on agents with handoffs. There’s a shared context and each agent has tools and the ability to transfer control to other agents. The original swarm concept was created by OpenAI.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Swarm

In [2]:
# from autogen import ConversableAgent, AfterWorkOption, initiate_swarm_chat, LLMConfig
from autogen import ConversableAgent, AfterWorkOption, initiate_swarm_chat

# llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# 1. Create our agents
planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

lesson_planner = ConversableAgent(
    name="planner_agent", llm_config=llm_config, system_message=planner_message
)

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Make sure you provide recommendations each time the plan is updated.
"""

lesson_reviewer = ConversableAgent(
    name="reviewer_agent", llm_config=llm_config, system_message=reviewer_message
)

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

In [3]:
# 2. Initiate the swarm chat using a swarm manager who will
# select agents automatically
result, _, _ = initiate_swarm_chat(
    initial_agent=teacher,
    agents=[lesson_planner, lesson_reviewer, teacher],
    messages="Today, let's introduce our kids to the solar system.",
    max_rounds=10,
    swarm_manager_args={"llm_config": llm_config},
    after_work=AfterWorkOption.SWARM_MANAGER
)

_User (to chat_manager):

Today, let's introduce our kids to the solar system.

--------------------------------------------------------------------------------

Next speaker: teacher_agent


>>>>>>>> USING AUTO REPLY...
teacher_agent (to chat_manager):

That sounds like a fantastic idea! Introducing the solar system can be both exciting and educational for the kids. Here’s a lesson plan outline for teaching about the solar system:

### Lesson Plan: Introduction to the Solar System

**Grade Level:** 3rd-5th Grade  
**Duration:** 1 hour

#### Objectives:
- Students will be able to identify and name the planets in our solar system.
- Students will understand the concept of orbits and how planets move around the sun.
- Students will learn a few interesting facts about each planet.

#### Materials Needed:
- Poster of the solar system
- Model of the solar system (can be a mobile or a 3D model)
- Internet access for videos or presentations
- Worksheets for note-taking or a solar system fact 

Replying as _User. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


In [4]:
# print("Final Lesson Plan:\n", result.summary)
print(result.summary)

### Final Revised Lesson Plan: Introduction to the Solar System

**Grade Level:** 4th Grade  
**Duration:** 1 hour

#### Objectives:
- Students will be able to identify and name the planets in our solar system.
- Students will understand the concept of orbits and how planets move around the sun.
- Students will learn about the relationship between the sun and the earth, including day/night cycles and seasonal changes due to the tilt of the Earth’s axis.
- Students will collaborate in groups to create a scaled model of the solar system and write a brief narrative from the perspective of their assigned planet.

#### Materials Needed:
- Poster of the solar system
- Model of the solar system (can be a mobile or a 3D model)
- Internet access for videos or presentations
- Worksheets for note-taking or a solar system fact sheet
- Art supplies for the solar system model (colored paper, markers, scissors, glue)
- Access to computers or tablets for coding simulation (such as Scratch or Tinkercad

## Controlling swarm transitions

In [ ]:
from autogen import (
    AfterWork,
    OnCondition,
    AfterWorkOption,
    ConversableAgent,
    SwarmResult,
    initiate_swarm_chat,
    register_hand_off,
    # LLMConfig,
)

# llm_config = LLMConfig(api_type="openai", model="gpt-4o", cache_seed=None)
llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}

# 1. Context
shared_context = {
    "lesson_plans": [],
    "lesson_reviews": [],
    # Will be decremented, resulting in 0 (aka False) when no reviews are left
    "reviews_left": 2,
}


# 2. Functions
def record_plan(lesson_plan: str, context_variables: dict) -> SwarmResult:
    """Record the lesson plan"""
    context_variables["lesson_plans"].append(lesson_plan)

    # Returning the updated context so the shared context can be updated
    return SwarmResult(context_variables=context_variables)


def record_review(lesson_review: str, context_variables: dict) -> SwarmResult:
    """After a review has been made, increment the count of reviews"""
    context_variables["lesson_reviews"].append(lesson_review)
    context_variables["reviews_left"] -= 1

    # Controlling the flow to the next agent from a tool call
    return SwarmResult(
        agent=teacher if context_variables["reviews_left"] < 0 else lesson_planner, context_variables=context_variables
    )


planner_message = """You are a classroom lesson planner.
Given a topic, write a lesson plan for a fourth grade class.
If you are given revision feedback, update your lesson plan and record it.
Use the following format:
<title>Lesson plan title</title>
<learning_objectives>Key learning objectives</learning_objectives>
<script>How to introduce the topic to the kids</script>
"""

# 3. Our agents now have tools to use (functions above)
lesson_planner = ConversableAgent(
    name="planner_agent", llm_config=llm_config, system_message=planner_message, functions=[record_plan]
)

reviewer_message = """You are a classroom lesson reviewer.
You compare the lesson plan to the fourth grade curriculum
and provide a maximum of 3 recommended changes for each review.
Always provide feedback for the current lesson plan.
"""

lesson_reviewer = ConversableAgent(
    name="reviewer_agent", llm_config=llm_config, system_message=reviewer_message, functions=[record_review]
)

teacher_message = """You are a classroom teacher.
You decide topics for lessons and work with a lesson planner.
and reviewer to create and finalise lesson plans.
"""

teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

# 4. Transitions using hand-offs

# Lesson planner will create a plan and hand off to the reviewer if we're still
# allowing reviews. After that's done, transition to the teacher.
register_hand_off(lesson_planner,
    [
        OnCondition(
            target=lesson_reviewer,
            condition="After creating/updating and recording the plan, it must be reviewed.",
            available="reviews_left",
        ),
        AfterWork(agent=teacher),
    ]
)

# Lesson reviewer will review the plan and return control to the planner if there's
# no plan to review, otherwise it will provide a review and
register_hand_off(lesson_reviewer,
    [
        OnCondition(
            target=lesson_planner, condition="After new feedback has been made and recorded, the plan must be updated."
        ),
        AfterWork(agent=teacher),
    ]
)

# Teacher works with the lesson planner to create a plan. When control returns to them and
# a plan exists, they'll end the swarm.
register_hand_off(teacher,
    [
        OnCondition(target=lesson_planner, condition="Create a lesson plan.", available="reviews_left"),
        AfterWork(AfterWorkOption.TERMINATE),
    ]
)

In [ ]:
# 5. Run the Swarm which returns the chat and updated context variables
chat_result, context_variables, last_agent = initiate_swarm_chat(
    initial_agent=teacher,
    agents=[lesson_planner, lesson_reviewer, teacher],
    messages="Today, let's introduce our kids to the solar system.",
    context_variables=shared_context,
)

In [ ]:
print(f"Number of reviews: {len(context_variables['lesson_reviews'])}")
print(f"Reviews remaining: {context_variables['reviews_left']}")
print(f"Final Lesson Plan:\n{context_variables['lesson_plans'][-1]}")